In [80]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import librosa
import sklearn
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.preprocessing import StandardScaler
from matplotlib import style
import pandas
from sklearn.ensemble import RandomForestClassifier as RFC
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import csv
from sklearn import svm
import itertools
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import pandas as pd
from scipy.special import comb
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif, mutual_info_classif, SelectPercentile, GenericUnivariateSelect
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from scipy.stats import rankdata

In [92]:
DATADIR = "/Users/yash_/Documents/project/t_overlap=0.25"
CATEGORIES = ["Segmented_Laugh", "Segmented_NonLaugh"]

# Defining Functions

In [93]:
def createcombinations(n):
    elements = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
    a = list(itertools.combinations(elements, n))
    a = np.asarray(a).astype(int)
    return a

In [94]:
def create_training_data(t_frame, t_shift, n_mfccs, component, training_data, n_features):
    count=0
    error_count = 0
    laugh_counter = 0
    nonlaugh_counter = 0
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category) 
        class_num = CATEGORIES.index(category)
        for aud in os.listdir(path):
            if aud == '.DS_Store':
                continue
            
            aud_array , sr = librosa.load(os.path.join(path,aud), sr=None)
            count+=1

            mfccs = []
            
            try:
                mfcc = (librosa.feature.mfcc(aud_array, sr=sr,  n_mfcc=20,  win_length = int(sr*t_frame), hop_length = int(sr*t_shift))) 
                mfcc_temp = mfcc[component,:]
                mfcc_temp_delta = librosa.feature.delta(mfcc_temp)
                mfcc_temp_delta2 = librosa.feature.delta(mfcc_temp)
                mean_mfccs = np.mean(np.asarray(mfcc_temp),axis = 1)
                std_mfccs = np.std(np.asarray(mfcc_temp),axis = 1)
                std_mfccs_delta = np.std(np.asarray(mfcc_temp_delta),axis = 1)
                std_mfccs_delta2 = np.std(np.asarray(mfcc_temp_delta2),axis = 1)
                
                #var_mfccs = np.var(np.asarray(mfcc_temp), axis = 1)
                
                mfccs.append(mean_mfccs)
                mfccs.append(std_mfccs)
                mfccs.append(std_mfccs_delta)
                mfccs.append(std_mfccs_delta2)
                #mfccs.append(var_mfccs)
                mfccs = np.asarray(mfccs).reshape(n_mfccs*n_features,1)
                
                training_data.append([mfccs.reshape(-1,1), class_num])
                if category == 'Segmented_Laugh':
                    laugh_counter +=1
                else:
                    nonlaugh_counter += 1
                    if nonlaugh_counter >= laugh_counter:
                        break

            except ValueError:
                pass
            
    return laugh_counter, nonlaugh_counter
    

# Main Loop

In [95]:
def scaling(X,n_mfcc):
    for i in range(n_mfcc):
        X[:,i] = X[:,i]/(np.mean(X[:,i]))
        

    #X[:,1] = np.square(X[:,1])
        
    return X

In [96]:
n_features = 4
t_frame = 0.025
t_shift = 0.01
n_mfcc = 20
component = createcombinations(n_mfcc)

for components in component:
    training_data = []
    laugh, nonlaugh = create_training_data(t_frame, t_shift, n_mfcc, components, training_data, n_features)
    print(laugh, nonlaugh)
    
random.shuffle(training_data) 
X = []
Y = []
for features, label in training_data:
    X.append(features)
    Y.append(label)
        
X = np.array(X).reshape(-1,n_mfcc*n_features)
Y = np.array(Y)
X = scaling(X,n_mfcc)

#X_train, X_test, y_train, y_test = train_test_split(
         #X, Y, test_size=0.2, random_state=123)

2704 2704


 #Using SFS ( is forward = True ) and SBS ( if forward = False)

In [ ]:

knn = svm.SVC(kernel='linear')

sfs1 = SFS(estimator=knn, 
           k_features=20, 
           forward=True, # if forward = True then SFS otherwise SBS
           floating=False, 
           verbose=2,
           scoring='accuracy'
       )


sfs1 = sfs1.fit(X, Y)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s


In [ ]:
df = pd.DataFrame.from_dict(sfs1.get_metric_dict()).T

####### Run Till Here ########

In [ ]:

index_0 = []
index_1 = []
result = []
for i in range(len(gs.cv_results_['params'])):
    dic = gs.cv_results_['params'][i].values()
    lis = list(dic)
    index_0.append(int(lis[0]))
    result.append(float(gs.cv_results_['mean_test_score'][i]))
    index_1.append(int(lis[1]))
    


In [ ]:
dataframe  = pd.DataFrame(columns=['sfs__estimator__n_neighbors','sfs__k_features','Test Accuracy'])
dataframe['sfs__estimator__n_neighbors'] = np.array(index_0)
dataframe['sfs__k_features'] = np.array(index_1)
dataframe['Test Accuracy'] = np.array(result)

In [ ]:
pipe.set_params(**gs.best_params_).fit(X_train, y_train)
print('Best features:', pipe.steps[0][1].k_feature_idx_)

In [ ]:
a = dataframe.to_csv('/Users/yash_/Documents/project/file1.csv')


In [ ]:
a

In [ ]:
style.use('dark_background')
fig1 = plot_sfs(sfs1.get_metric_dict(), kind='std_dev')
plt.title('Sequential Backward Selection (w. StdDev)')
#plt.grid()
plt.show()


##########################################################################################################

In [ ]:
test = SelectKBest(score_func=f_classif, k=5)
fit = test.fit(np.abs(X), Y)
np.set_printoptions(precision=3)
fclass = fit.scores_

test = SelectKBest(score_func=mutual_info_classif, k=5)
fit = test.fit(np.abs(X), Y)
np.set_printoptions(precision=3)
mutual = fit.scores_

In [ ]:
row_list = [["MFCCs", 'f_classif Score', 'Mutual Info Classif Score']]
for i in range(20):
    row_list.append([str(i) + ' delta2', np.int(fclass[i]), np.float(mutual[i])])
    
with open('MFCC_Delta2_' + str(n_mfcc) + '.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(row_list)



In [ ]:
plt.plot(fit.scores_,'*')

In [ ]:
rankdata(fit.scores_)

In [ ]:
mean = []
meanlaugh = []
meannon = []
std = []
for i in range(20):
    mean.append(np.mean(X[:,i]))
    meanlaugh.append(np.mean(X[np.where(Y==0),i]))
    meannon.append(np.mean(X[np.where(Y==1),i]))
    std.append(np.std(X[:,i]))
    print('i = ', i, 'Mean = ', mean[i], 'Laugh =', meanlaugh[i], 'Non=', meannon[i])

In [ ]:
plt.figure(figsize=(20, 12))
plt.plot(mean,'ko')
plt.plot(meanlaugh,'r*')
plt.plot(meannon,'r*')
plt.plot(np.array(mean)+np.array(std),'b.')
plt.plot(np.array(mean)-np.array(std),'b.')

In [ ]:
plt.plot((X[np.where(Y==1),14]).T,'.')
plt.plot((X[np.where(Y==0),14]).T,'.')

In [ ]:
plt.plot(np.square(X[np.where(Y==1),12]).T,'.')
plt.plot(np.square(X[np.where(Y==0),12]).T,'.')

In [ ]:
plt.plot(X[np.where(Y==1),2].T,'.')
plt.plot(X[np.where(Y==0),2].T,'.')

plt.plot(X[np.where(Y==1),12].T,'.')
plt.plot(X[np.where(Y==0),12].T,'.')

In [ ]:
style.use('dark-background')
plt.plot(X[np.where(Y==1),1].T,'.')
plt.plot(X[np.where(Y==0),0].T,'.')